In [2]:
import sys
import os
sys.path.append(os.path.abspath("../.."))
import numpy as np
import yfinance as yf
import plotly.express as px
from tinyshift.series import *
from tinyshift.stats import trailing_window, mad
from numpy.random import standard_normal

In [3]:
ticker = 'PETR4.SA'
period = '2y'
data = yf.download(ticker, period=period, interval="1d", auto_adjust=True)
t = np.linspace(0, 10, 1000)
pure_sine = np.sin(2 * np.pi * 1 * t)  # Perfectly forecastable

[*********************100%***********************]  1 of 1 completed


In [4]:
r = standardize_returns(data["Close"]["PETR4.SA"], log=False)

# RSI

In [5]:
vol_rsi = relative_strength_index(data["Close"]["PETR4.SA"], rolling_window=14)

In [6]:
px.line(vol_rsi, title=f"{ticker} Closing Prices").show(fig_type="png")

# Entropy Volatility

In [7]:
vol_ent = trailing_window(r, rolling_window=60, func=sample_entropy, m=1)

In [8]:
px.line(vol_ent, title=f"{ticker} Closing Prices").show(fig_type="png")

In [9]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    y=data["Close"]["PETR4.SA"],
    x=data.index,
    name="Preço PETR4.SA",
    yaxis="y1"
))

fig.add_trace(go.Scatter(
    x=data.index,
    y=vol_ent,
    name="Volatility Entropy",
    yaxis="y2"
))

fig.add_trace(go.Scatter(
    x=data.index,
    y=vol_rsi,
    name="Volatility RSI",
    yaxis="y3"
))


fig.update_layout(
    yaxis3=dict(
        title="Volatility RSI",
        anchor="free",
        overlaying="y",
        side="right",
        position=1.0,
        showgrid=False
    )
)

fig.update_layout(
    title="Preço PETR4.SA e Volatility Entropy",
    xaxis_title="Data",
    yaxis=dict(title="Preço PETR4.SA"),
    yaxis2=dict(title="Volatility Entropy", overlaying="y", side="right"),
    legend=dict(x=0.01, y=0.99)
)

fig.show(fig_type="png")

# Bollinger's Band

In [10]:
np.sum(bollinger_bands(data["Close"]["PETR4.SA"]))

np.int64(52)

In [11]:
np.sum(bollinger_bands(data["Close"]["PETR4.SA"], center=np.median, spread=mad))

np.int64(171)

# Hampel Filter

In [12]:
hf = hampel_filter(data["Close"]["PETR4.SA"], rolling_window=7)

In [13]:
data["Close"]["PETR4.SA"][hf]

Date
2024-05-22    30.161886
2024-07-04    31.138037
2024-10-09    31.938402
2025-02-07    33.884270
Name: PETR4.SA, dtype: float64

# ADI-CV Framework

In [14]:
adi_cv(r)

(np.float64(1.0), np.float64(8.803129168251592e+33))

# Sampen

In [15]:
sample_entropy(r, m=3)

np.float64(1.6244757897986408)

In [16]:
sample_entropy(pure_sine, m=2)

np.float64(0.16791509477225003)

# Permutation Entropy

In [31]:
permutation_entropy(r, m=3, delay=1, normalize=False)

np.float64(2.58405472798716)

In [32]:
permutation_entropy(r, m=3, delay=1, normalize=True)

np.float64(0.9996488255695226)

In [35]:
permutation_entropy(data["Close"]["PETR4.SA"], m=3, delay=1, normalize=False)

np.float64(2.4855077334923203)

# Pattern Stability Index

In [56]:
pattern_stability_index(r)

np.float64(0.1364091685816982)

In [57]:
pattern_stability_index(pure_sine)

np.float64(0.8656234267494547)

# Maximum Achievable Predictability

In [54]:
maximum_achievable_predictability(data["Close"]["PETR4.SA"], m=4, delay=1)

np.float64(0.0779381740053422)

In [55]:
maximum_achievable_predictability(pure_sine, m=4, delay=1)

np.float64(0.7114310780157527)

# Foreca

In [18]:
foreca(r)

0.07349910737780474

In [19]:
trailing_window(r, rolling_window=60, func=foreca)

array([0.14383021, 0.14383021, 0.14383021, 0.14383021, 0.14383021,
       0.14383021, 0.14383021, 0.14383021, 0.14383021, 0.14383021,
       0.14383021, 0.14383021, 0.14383021, 0.14383021, 0.14383021,
       0.14383021, 0.14383021, 0.14383021, 0.14383021, 0.14383021,
       0.14383021, 0.14383021, 0.14383021, 0.14383021, 0.14383021,
       0.14383021, 0.14383021, 0.14383021, 0.14383021, 0.14383021,
       0.14383021, 0.14383021, 0.14383021, 0.14383021, 0.14383021,
       0.14383021, 0.14383021, 0.14383021, 0.14383021, 0.14383021,
       0.14383021, 0.14383021, 0.14383021, 0.14383021, 0.14383021,
       0.14383021, 0.14383021, 0.14383021, 0.14383021, 0.14383021,
       0.14383021, 0.14383021, 0.14383021, 0.14383021, 0.14383021,
       0.14383021, 0.14383021, 0.14383021, 0.14383021, 0.14383021,
       0.13523003, 0.13534289, 0.13263972, 0.14032888, 0.13470074,
       0.12967773, 0.12326279, 0.12346414, 0.11431565, 0.12105077,
       0.12100532, 0.11642939, 0.12136623, 0.12150759, 0.12041

In [20]:
foreca(pure_sine)

0.9993976808641186

In [21]:
trailing_window(pure_sine, rolling_window=60, func=foreca)

array([0.87814292, 0.87814292, 0.87814292, 0.87814292, 0.87814292,
       0.87814292, 0.87814292, 0.87814292, 0.87814292, 0.87814292,
       0.87814292, 0.87814292, 0.87814292, 0.87814292, 0.87814292,
       0.87814292, 0.87814292, 0.87814292, 0.87814292, 0.87814292,
       0.87814292, 0.87814292, 0.87814292, 0.87814292, 0.87814292,
       0.87814292, 0.87814292, 0.87814292, 0.87814292, 0.87814292,
       0.87814292, 0.87814292, 0.87814292, 0.87814292, 0.87814292,
       0.87814292, 0.87814292, 0.87814292, 0.87814292, 0.87814292,
       0.87814292, 0.87814292, 0.87814292, 0.87814292, 0.87814292,
       0.87814292, 0.87814292, 0.87814292, 0.87814292, 0.87814292,
       0.87814292, 0.87814292, 0.87814292, 0.87814292, 0.87814292,
       0.87814292, 0.87814292, 0.87814292, 0.87814292, 0.87814292,
       0.85659439, 0.83599743, 0.81701635, 0.79995579, 0.78488091,
       0.77171463, 0.76030651, 0.75047711, 0.74204419, 0.73483677,
       0.72870169, 0.72350584, 0.7191359 , 0.71549702, 0.71251

# Hurst's Exponent

In [22]:
hurst_exponent(r)

(0.4297226522891123, 0.15931663643759408)

In [23]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    y=data["Close"]["PETR4.SA"],
    x=data.index,
    name="Preço PETR4.SA",
    yaxis="y1"
))


fig.show(fig_type="png")

In [24]:
np.random.seed(42)
n_points = 1000
trend = np.linspace(0, 10, n_points)
noise_trend = np.cumsum(standard_normal(n_points) * 0.1)
mean_reversion = np.zeros(n_points)
for t in range(1, n_points):
    mean_reversion[t] = mean_reversion[t-1] * 0.6 + standard_normal() * 0.5
brownian = np.cumsum(standard_normal(n_points))

In [25]:
hurst_exponent(noise_trend)

(0.6419241912818302, 0.000952023886777138)

In [26]:
hurst_exponent(mean_reversion)

(0.3685698852341455, 0.05402692976986088)

In [27]:
hurst_exponent(pure_sine)

(0.8575205638768405, 0.0014654668633357023)